In [1]:
from pf2barcode.imports import import_CCLE
from anndata import AnnData
import scanpy as sc
import pandas as pd

X = import_CCLE()

# Get rid of cells with no barcode
X = X[X.obs["SW"] != "unknown"]

# remove cells with barcodes having less than 10 cells
good_SW = X.obs["SW"].value_counts().index[X.obs["SW"].value_counts() > 10]
X = X[X.obs["SW"].isin(good_SW)]

/home/asm/.cache/pypoetry/virtualenvs/pf2barcode-Hbu_9TGN-py3.12/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [2]:
pcadata = AnnData(
    X.obsm["X_pca"],
    obs=X.obs,
    var=pd.DataFrame(index=[f"PC{i}" for i in range(1, X.obsm["X_pca"].shape[1] + 1)]),
)
pcadata.X /= pcadata.X.mean(axis=1, keepdims=True)

## Stats

In [6]:
from scipy.stats import f_oneway

comp = pcadata

In [7]:
from gseapy import prerank, dotplot

geneSets = [
    "GO_Biological_Process_2021",
    "GO_Cellular_Component_2021",
]

df = prerank(
    pd.Series(H[:, 19], index=X.var_names),
    gene_sets=geneSets,
    organism="Human",
    no_plot=True,
    threads=20,
)

dotplot(
    df.res2d,
    column="FDR q-val",
    title="KEGG_2016",
    size=6,  # adjust dot size
    figsize=(4, 5),
    cutoff=0.25,
    show_ring=False,
);

NameError: name 'H' is not defined

In [ ]:
df.res2d.loc[df.res2d["FDR q-val"] < 0.1, :]

In [ ]:
pd.Series(H[:, 20], index=X.var_names).sort_values(ascending=False).head(20)